In [1]:
from data.helper import *

In [2]:
DF = get_plan_df("pfcalcul")

100%|██████████| 46080/46080 [00:03<00:00, 15268.81it/s]


In [3]:
COLUMNS = [
    # "facts",
    "optimiser",
    "features",
    "iterations",
    "feature_pruning",
    "data_pruning",
]

In [4]:
def get_df_copy():
    df = DF.copy()
    # df = df[df.optimiser == "svr"]
    # df = df[df.pruning == "none"]
    # df = df[df.iterations == "4"]
    df = df[df.facts == "fd"]
    df = df[df.features == "wl"]
    return df

for metric in [
    # "tried",
    "solved",
]:
    print(metric)
    df = get_df_copy()
    # combine rows with same pruning, optimiser and iterations, and sum the solved, with sum values made into int
    df = df.groupby(COLUMNS + ["domain"]).sum().reset_index()
    df = df[COLUMNS + ["domain", metric]]
    # pivot table
    df = df.pivot_table(index=COLUMNS, columns="domain", values=metric)
    # add a column that sums all integer entries per row
    df["total"] = df.sum(axis=1)
    # make entries int
    df = df.astype(int)
    df = df.fillna(0)
    # display df without hiding rows
    with pd.option_context("display.max_rows", None):
        display(df)

solved


domain                                                       blocksworld  \
optimiser features iterations feature_pruning  data_pruning                
rank-svm  wl       2          collapse-all     equivalent             22   
                                               none                   24   
                              collapse-layer   equivalent             22   
                                               none                   24   
                              collapse-layer-x equivalent             15   
                                               none                   16   
                              none             equivalent             23   
                                               none                   23   
                   4          collapse-all     equivalent             24   
                                               none                   23   
                              collapse-layer   equivalent             21   
                                               none                   23   
                              collapse-layer-x equivalent             15   
                                               none                   16   
                              none             equivalent             21   
                                               none                   21   
                   6          collapse-all     equivalent             23   
                                               none                   24   
                              collapse-layer   equivalent             23   
                                               none                   23   
                              collapse-layer-x equivalent             15   
                                               none                   15   
                              none             equivalent              0   
                                               none                    0   
svr       wl       2          collapse-all     equivalent             11   
                                               none                   12   
                              collapse-layer   equivalent             10   
                                               none                   11   
                              collapse-layer-x equivalent             15   
                                               none                   16   
                              none             equivalent             11   
                                               none                   11   
                   4          collapse-all     equivalent             20   
                                               none                   19   
                              collapse-layer   equivalent             10   
                                               none                   10   
                              collapse-layer-x equivalent             15   
                                               none                   16   
                              none             equivalent             20   
                                               none                   21   
                   6          collapse-all     equivalent             20   
                                               none                   18   
                              collapse-layer   equivalent             12   
                                               none                   12   
                              collapse-layer-x equivalent             15   
                                               none                   16   
                              none             equivalent             20   
                                               none                   21   

domain                                                       childsnack  \
optimiser features iterations feature_pruning  data_pruning               
rank-svm  wl       2          collapse-all     eq

In [7]:
for metric in ["expanded", "runtime", "plan_length"]:
    htmls = ""
    for domain in DOMAINS:
        df = get_df_copy()
        df = df[df.domain == domain]
        # replace -1 values with Nan
        df = df.replace(-1, np.nan)
        df["config"] = df["features"] + " - " + df["iterations"] + " - " + df["optimiser"] + " - " + df["feature_pruning"]
        fig = px.line(
            df,
            x="problem",
            y=metric,
            color="config",
            # symbol="feature_pruning",
            # line_dash="feature_pruning",
            title=f"{domain} {metric}",
            width=800,
            height=600,
            log_y=True,
            markers=True,
        )
        htmls += fig.to_html()
    with open(f"{PLOT_DIR}/{metric}.html", "w") as f:
        f.write(htmls)